# Трекер для кастомной модели

In [13]:

import torch

# model = YOLO(self.model_path) 
# model.fuse()



model_name='best.pt'
model = torch.hub.load("D:/projects/otus/otus_cv/hw_final/yolov5", 'custom',
                        source='local', path = "D:/projects/otus/otus_cv/hw_final/yolov5/best.pt", force_reload=True)

type(model)
#model.fuse()

#model = torch.hub.load('D:/projects\otus\otus_cv\hw_final\weights','custom', path='best.pt',force_reload=True,source='local', pretrained = False)
#model = torch.hub.load('.', 'custom', 'yolov5s.pt', source='local')


YOLOv5  v7.0-366-gf7322921 Python-3.9.19 torch-2.4.1+cpu CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7254609 parameters, 0 gradients
Adding AutoShape... 


models.common.AutoShape

In [22]:
import torch
import numpy as np
import cv2
from time import time  
from ultralytics import YOLO

from sort import Sort

class ObjectDetection:

    def __init__(self, capture_dev, model_path, weights_name, height=1280, width=720):
        self.capture_dev = capture_dev
        self.model_path = model_path
        self.weights_name = weights_name
        self.height = height
        self.width = width

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)
        
        self.model = self.load_model()
        # self.CLASS_NAMES_DICT = self.model.model.names

    def load_model(self):
        #model = YOLO(self.model_path) 
        model = torch.hub.load(self.model_path, 'custom', source='local', path = self.weights_name, force_reload=True)

        #model.fuse()
        return model

    def predict(self, frame):
        #results = self.model(frame, verbose=False)
        results = self.model(frame)
        print(results)
        return results


# image 1/1: 360x640 2 tractors
# Speed: 2.2ms pre-process, 100.0ms inference, 1.0ms NMS per image at shape (1, 3, 384, 640)
#          xmin        ymin        xmax       ymax  confidence  class     name
# 0  319.510437  143.295914  370.080200  181.98262    0.655240      3  tractor
# 1  226.162857  152.792435  286.686249  188.32872    0.529719      3  tractor
    def get_results(self, results, img):
        detections_list = []

        data_frame = results.pandas().xyxy[0]
        print('data_frame:')
        print(data_frame)

        # Get indexes of all of the rows
        indexes = data_frame.index
        for index in indexes:
            # Find the coordinate of top left corner of bounding box
            x1 = int(data_frame['xmin'][index])
            y1 = int(data_frame['ymin'][index])
            # Find the coordinate of right bottom corner of bounding box
            x2 = int(data_frame['xmax'][index])
            y2 = int(data_frame['ymax'][index ])

            # Find label name
            label = data_frame['name'][index ]
            # Find confidance score of the model
            conf = data_frame['confidence'][index]
            text = label + ' ' + str(conf.round(decimals= 2))

            merged_detection = [x1, y1, x2, y2, conf]
            #[bbox[0][0], bbox[0][1], bbox[0][2], bbox[0][3], confidence[0]]
            detections_list.append(merged_detection)

        return np.array(detections_list)

    def draw_bounding_boxes_with_id(self, img, bboxes, ids):
        for bbox, id_ in zip(bboxes, ids):
            cv2.rectangle(img,(int(bbox[0]), int(bbox[1])),(int(bbox[2]), int(bbox[3])),(0,0,255),2)
            cv2.putText(img, "ID: " + str(id_), (int(bbox[0]), int(bbox[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        return img

    def __call__(self):
        cap = cv2.VideoCapture(self.capture_dev)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.height)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.width)

        # SORT
        sort = Sort(max_age=100, min_hits=8, iou_threshold=0.50)

        while True:

            start_time = time()
            ret, frame = cap.read()
            assert ret

            results = self.predict(frame)
            detections_list = self.get_results(results, frame)

            # # SORT Tracking
            if len(detections_list) == 0:
                detections_list = np.empty((0, 5))

            res = sort.update(detections_list)

            # boxes_track = res[:,:-1]
            # boxes_ids = res[:,-1].astype(int)

            # frame = self.draw_bounding_boxes_with_id(frame, boxes_track, boxes_ids)

            # end_time = time()
            # fps = 1/np.round(end_time - start_time, 2)

            # cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

            # cv2.imshow('YOLOv8 Detection', frame)
 
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
            break
        cap.release()
        cv2.destroyAllWindows()


In [23]:
import os
YOLO_DIR = os.getcwd() + '/yolov5'

detector = ObjectDetection(capture_dev="./videos/tracktor_competition_360p.mp4",
                           model_path=YOLO_DIR, 
                           weights_name=YOLO_DIR+"/best.pt")
detector()

YOLOv5  v7.0-366-gf7322921 Python-3.9.19 torch-2.4.1+cpu CPU

Fusing layers... 


Using Device:  cpu


custom_YOLOv5s summary: 182 layers, 7254609 parameters, 0 gradients
Adding AutoShape... 
d:\projects\otus\otus_cv\hw_final/yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


image 1/1: 360x640 2 tractors
Speed: 2.0ms pre-process, 104.9ms inference, 1.0ms NMS per image at shape (1, 3, 384, 640)
data_frame:
         xmin        ymin        xmax       ymax  confidence  class     name
0  319.510437  143.295914  370.080200  181.98262    0.655240      3  tractor
1  226.162857  152.792435  286.686249  188.32872    0.529719      3  tractor


: 